### Projeto simples de um sostema bancario 

--> funções principais:
    Sacar com limite de 3 saques diarios, cada saque de no máximo 500 reais
    depositar 
    Extrato --> com todas movimentações.

In [3]:
# Variáveis globais
saldo = 0
limite_saque = 500
saques_realizados = 0
limite_saques_diarios = 3
extrato = []

def depositar(valor):
    global saldo
    if valor > 0:
        saldo += valor
        extrato.append(f"Depósito: +R$ {valor:.2f}")
        print(f"Depósito de R$ {valor:.2f} realizado com sucesso.")
    else:
        print("Valor inválido para depósito.")

def sacar(valor):
    global saldo, saques_realizados
    if saques_realizados >= limite_saques_diarios:
        print("Limite diário de saques atingido.")
    elif valor > limite_saque:
        print(f"Limite por saque é de R$ {limite_saque:.2f}.")
    elif valor > saldo:
        print("Saldo insuficiente.")
    elif valor <= 0:
        print("Valor inválido para saque.")
    else:
        saldo -= valor
        saques_realizados += 1
        extrato.append(f"Saque: -R$ {valor:.2f}")
        print(f"Saque de R$ {valor:.2f} realizado com sucesso.")

def mostrar_extrato():
    print("\n📄 Extrato Bancário:")
    if not extrato:
        print("Nenhuma movimentação realizada.")
    else:
        for operacao in extrato:
            print(operacao)
    print(f"\nSaldo atual: R$ {saldo:.2f}\n")

# Exemplo de uso
depositar(1000)
sacar(200)
sacar(300)
sacar(100)  # Este será bloqueado por limite de saques
mostrar_extrato()

Depósito de R$ 1000.00 realizado com sucesso.
Saque de R$ 200.00 realizado com sucesso.
Saque de R$ 300.00 realizado com sucesso.
Saque de R$ 100.00 realizado com sucesso.

📄 Extrato Bancário:
Depósito: +R$ 1000.00
Saque: -R$ 200.00
Saque: -R$ 300.00
Saque: -R$ 100.00

Saldo atual: R$ 400.00

